# Modelado de la degradación del desempeño de una turbomáquina empleando aprendizaje automático.

###  Data 

Los conjuntos de datos utilizados en este __Trabaja Especial de Grado__ reúnen toda la información necesaria. Una lista de publicaciones científicas relacionadas está disponible gratuitamente en el repositorio de datos de Ames Prognostics de la NASA. [Nasa](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/)

<img src="https://www.researchgate.net/profile/Jonathan_Decastro2/publication/265996485/figure/fig1/AS:392196856532995@1470518425804/Simplified-diagram-of-the-90K-engine.png">


| Data Set | Máquinas de entrenamiento | Máquinas de prueba | Condiciones      | Modo de falla          |
|----------|:-----------------:|------------------:|:---------------:|:--------------------------------------:|
| FD001    |        100        |               100 | ONE (Sea Level) | ONE (HPC Degradation)                  |
| FD002    |        260        |               259 | SIX             | ONE (HPC Degradation)                  |
| FD003    |        100        |               100 | ONE (Sea Level) | TWO (HPC Degradation, Fan Degradation) |
| FD004    |        249        |               248 | SIX             | TWO (HPC Degradation, Fan Degradation) |

La información acerca de las condiciones de operación se muestra en la siguiente tabla:

| Cond. Op. | Altitud (ft) Opsett1 |Mach Opsett2|TRA (%) Opsett3|  EPR (10) | Nf (rpm) (8) | Nc (rpm) (9) | Ps30 (ºR) (11) | 
|------------------------|:------------:|------------:|:-------:|:----------|-------------:|-------------:|---------------:|
| ONE (Sea Level)        |   aprox. 0   |  aprox. 0   |   100   |    1.30   |    2388.00   |   9051.00    |     522.13     |
| SIX                    |    (0-42K)   |  (0-0.84)   |   100   |    1.02   |    2212.00   |   8317.00    |     130.51     | 

<img src="http://oi67.tinypic.com/10z5yyp.jpg">

## Escenario experimental

Los conjuntos de datos consisten en múltiples series de tiempo. Cada conjunto de datos se divide en subgrupos de entrenamiento y prueba. Cada serie temporal es de un motor diferente. Cada motor arranca con diferentes grados de desgaste inicial y variación de fabricación que es desconocida para el usuario. Este desgaste y variación inicial se consideran normales, es decir, no se considera una condición de falla. Hay tres configuraciones operativas que tienen un efecto sustancial en el rendimiento del motor. Estas configuraciones también se incluyen en los datos. Los datos están contaminados con el ruido del sensor.

* Datos de entrenamiento: son los datos de ejecución del motor del avión desde el estado inicial hasta al fallo.
* Datos de prueba: Son los datos de operación del motor de la aeronave sin eventos de falla registrados.
* Datos RUL: contiene la información de los ciclos verdaderos restantes para cada motor en los datos de prueba.

Los datos de entrenamiento __("train_FD001.txt")__ consisten en múltiples series de tiempo con "ciclos" como unidad de tiempo, junto con 21 lecturas de sensores para cada ciclo. Se puede suponer que cada serie de tiempo se genera a partir de un motor diferente del mismo tipo. Además, también se supone que cada motor comienza con diferentes grados de desgaste inicial y variación de fabricación, y esta información es desconocida para el usuario.

__En estos datos de entrenamiento__, se supone que el motor está funcionando normalmente al comienzo de cada serie de tiempo, luego, comienza a degradarse en algún momento durante la serie de los ciclos operativos. La degradación progresa y crece en magnitud. Cuando se alcanza un umbral predefinido, entonces el motor se considera inseguro para una operaciones posteriores. En otras palabras, __el último ciclo en cada serie de tiempo puede considerarse como el punto de falla del motor correspondiente.__

Tomando como ejemplo los datos de entrenamiento, el motor con id = 1 falla en el ciclo 192, y el motor con id = 2 falla en el ciclo 287.

Los datos de prueba __("test_FD001.txt")__ tienen el mismo esquema que los datos de entrenamiento. La única diferencia es que __los datos no indican cuándo ocurre la falla (en otras palabras, el último período de tiempo NO representa el punto de falla).__ Tomando como ejemplo los datos de prueba, el motor con id = 1 funciona desde el ciclo 1 hasta el ciclo 31. No se muestran cuántos ciclos más puede durar este motor antes de que falle.

Los datos de vida útil remanente __("RUL_FD001.txt")__ __proporciona el número de ciclos de trabajo restantes para los motores en los datos de prueba.__ Tomando como ejemplo los datos RUL, el motor con id = 1 en los datos de prueba puede ejecutar otros 112 ciclos antes de que falle.

* Basándonos en la descripción de datos de entrada que hemos examinado en los parrafos anteriores, __una pregunta intuitiva de mantenimiento predictivo es: "Teniendo en cuenta el funcionamiento de este motor de avión y el historial de eventos de fallas, ¿Podemos predecir cuándo fallará un motor en servicio?"__

<img src="http://oi67.tinypic.com/vq4k5i.jpg">


|Column|           Trayectorias              |                         Descripción                          | Unidades |
|-----:|:-----------------------------------:|:------------------------------------------------------------:|:--------:|
| 1    |        unit number                  | Número correspondiente a la máquina                          | --       |
| 2    |        time, in cycles              | Ciclo de operación de la máquina                             | --       |   
| 3    |        operational setting 1        | Altitud                                                      | ft       |
| 4    |        operational setting 2        | Número de Mach                                               | --       |
| 5    |        operational setting 3        | Ángulo de la palanca de aceleración del piloto __(TRA)__     | %        |  
| 6    |        sensor measurement  1        | Temperatura total a la entrada del ventilador __(T20)__      | °R       |
| 7    |        sensor measurement  2        | Temperatura total a la salida del LPC __(T24)__              | °R       |
| 8    |        sensor measurement  3        | Temperatura total a la salida del HPC __(T30)__              | °R       | 
| 9    |        sensor measurement  4        | Temperatura total a la salida de la LPT __(T50)__            | °R       | 
| 10   |        sensor measurement  5        | Presión a la entrada del ventilador __(P20)__                | psia     |
| 11   |        sensor measurement  6        | Presión total en el ducto de Bypass __(P15)__                | psia     | 
| 12   |        sensor measurement  7        | Presión total a la salida del HPC __(P30)__                  | psia     |
| 13   |        sensor measurement  8        | Velocidad física del ventilador __(Nf)__                     | rpm      | 
| 14   |        sensor measurement  9        | Velocidad física del núcleo __(Nc)__                         | rpm      |   
| 15   |        sensor measurement  10       | Relación de presión del motor (P50/P20) __(epr)__            | --       |   
| 16   |        sensor measurement  11       | Presión estática en el HPC __(Ps30)__                        | psia     |
| 17   |        sensor measurement  12       | Relación de flujo de combustible a Ps30 __(phi)__            | pps/psi  | 
| 18   |        sensor measurement  13       | Velocidad del ventilador corregida __(NRf)__                 | rpm      | 
| 19   |        sensor measurement  14       | Velocidad del núcleo corrediga __(NRc)__                     | rpm      | 
| 20   |        sensor measurement  15       | Relación de Bypass __(BPR)__                                 | --       | 
| 21   |        sensor measurement  16       | Relación combustible-aire del quemador __(farB)__            | --       |
| 22   |        sensor measurement  17       | Entalpia de purga __(htBleed)__                              | --       |
| 23   |        sensor measurement  18       | Velocidad exigida del ventilador __(Nf_dmd)__                | rpm      |
| 24   |        sensor measurement  19       | Velocidad corregida exigida del ventilador __(PCNfR_dmd)__   | rpm      |
| 25   |        sensor measurement  20       | Purga de refrigerante de HPT __(W31)__                       | lbm/s    |
| 26   |        sensor measurement  21       | Purga de refrigerante de LPT __(W32)__                       | lbm/s    |         

Referencias: 

1. A. Saxena, K. Goebel, D. Simon, and N. Eklund, "Damage Propagation Modeling for Aircraft Engine Run-to-Failure Simulation", in the Proceedings of the Ist International Conference on Prognostics and Health Management (PHM08), Denver CO, Oct 2008., retrieved feb. 2016
2. NASA Ames Prognostics data repository, retrieved feb. 2016, http://ti.arc.nasa.gov/tech/dash/pcoe/prognostic-data-repository/

In [1]:
import pandas as pd
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

# Lectura de la data

In [2]:
import pandas as pd

In [3]:
# Como los archivos de entrada no tienen nombres de columnas, se les agregaran ahora

sensors = ["sensorMeasure" + str(i) for i in range(1, 22)]
settings = ["operatingSet" + str(i) for i in range(1, 4)]
column_names = ["unitNumber", "cycle"] + settings + sensors

train_fd001 = pd.read_csv('CMAPSS_Data/train_FD001.txt', sep=' ', header=None)
test_fd001 = pd.read_csv('CMAPSS_Data/test_FD001.txt', sep=' ', header=None)
RUL_fd001 = pd.read_csv('CMAPSS_Data/RUL_FD001.txt', sep=' ', header=None)

train_fd002 = pd.read_csv("CMAPSS_Data/train_FD002.txt", sep=" ", header=None)
test_fd002 = pd.read_csv("CMAPSS_Data/test_FD002.txt", sep=" ", header=None)
RUL_fd002 = pd.read_csv("CMAPSS_Data/RUL_FD002.txt", sep=" ", header=None)

train_fd003 = pd.read_csv("CMAPSS_Data/train_FD003.txt", sep=" ", header=None)
test_fd003 = pd.read_csv("CMAPSS_Data/test_FD003.txt", sep=" ", header=None)
RUL_fd003 = pd.read_csv("CMAPSS_Data/RUL_FD003.txt", sep=" ", header=None)

train_fd004 = pd.read_csv("CMAPSS_Data/train_FD004.txt", sep=" ", header=None)
test_fd004 = pd.read_csv("CMAPSS_Data/test_FD004.txt", sep=" ", header=None)
RUL_fd004 = pd.read_csv("CMAPSS_Data/RUL_FD004.txt", sep=" ", header=None)

In [4]:
# Se puede verificar la data que se tiene

train_fd001.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 28 columns):
0     20631 non-null int64
1     20631 non-null int64
2     20631 non-null float64
3     20631 non-null float64
4     20631 non-null float64
5     20631 non-null float64
6     20631 non-null float64
7     20631 non-null float64
8     20631 non-null float64
9     20631 non-null float64
10    20631 non-null float64
11    20631 non-null float64
12    20631 non-null float64
13    20631 non-null float64
14    20631 non-null float64
15    20631 non-null float64
16    20631 non-null float64
17    20631 non-null float64
18    20631 non-null float64
19    20631 non-null float64
20    20631 non-null float64
21    20631 non-null int64
22    20631 non-null int64
23    20631 non-null float64
24    20631 non-null float64
25    20631 non-null float64
26    0 non-null float64
27    0 non-null float64
dtypes: float64(24), int64(4)
memory usage: 4.4 MB


In [5]:
test_fd001.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13096 entries, 0 to 13095
Data columns (total 28 columns):
0     13096 non-null int64
1     13096 non-null int64
2     13096 non-null float64
3     13096 non-null float64
4     13096 non-null float64
5     13096 non-null float64
6     13096 non-null float64
7     13096 non-null float64
8     13096 non-null float64
9     13096 non-null float64
10    13096 non-null float64
11    13096 non-null float64
12    13096 non-null float64
13    13096 non-null float64
14    13096 non-null float64
15    13096 non-null float64
16    13096 non-null float64
17    13096 non-null float64
18    13096 non-null float64
19    13096 non-null float64
20    13096 non-null float64
21    13096 non-null int64
22    13096 non-null int64
23    13096 non-null float64
24    13096 non-null float64
25    13096 non-null float64
26    0 non-null float64
27    0 non-null float64
dtypes: float64(24), int64(4)
memory usage: 2.8 MB


In [6]:
RUL_fd001.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
0    100 non-null int64
1    0 non-null float64
dtypes: float64(1), int64(1)
memory usage: 1.6 KB


**Todos los datos en cada columna son valores númericos. De igual forma, no hay valores faltantes (NaN) en ningún data set.**

**Para el caso del train y test las dos últimas columnas no tiene datos y para RUL solo la última no tiene datos. Entonces es bueno eliminarlas pero es una buena practica no eliminar las variables originales de los datos.**

# Arreglo de los datos

In [7]:
def acomodar_datos(datos, ncolumnas, colnames):
    """
    Esta función arregla los datos del dataset para poder usarlo
    en el analisis subsiguiente
    input: Datos: Son los datos del dataset
           ncolumnas: Corresponde al numero de columnas con valores perdidos
           colnames: Seria el nombre de las columnas
    """
    ncolumnas = -ncolumnas
    
    df = pd.DataFrame(data=datos.iloc[:,:ncolumnas].values, columns=colnames)
        
    return df

In [8]:
df_train_fd001 = acomodar_datos(train_fd001, 2, column_names)
df_test_fd001 = acomodar_datos(test_fd001, 2, column_names)
df_rul_fd001 = acomodar_datos(RUL_fd001, 1, ['RUL'])

df_train_fd002 = acomodar_datos(train_fd002, 2, column_names)
df_test_fd002 = acomodar_datos(test_fd002, 2, column_names)
df_rul_fd002 = acomodar_datos(RUL_fd002, 1, ['RUL'])

df_train_fd003 = acomodar_datos(train_fd003, 2, column_names)
df_test_fd003 = acomodar_datos(test_fd003, 2, column_names)
df_rul_fd003 = acomodar_datos(RUL_fd003, 1, ['RUL'])

df_train_fd004 = acomodar_datos(train_fd004, 2, column_names)
df_test_fd004 = acomodar_datos(test_fd004, 2, column_names)
df_rul_fd004 = acomodar_datos(RUL_fd004, 1, ['RUL'])

In [9]:
df_train_fd001.head()

,unitNumber,cycle,operatingSet1,operatingSet2,operatingSet3,sensorMeasure1,sensorMeasure2,sensorMeasure3,sensorMeasure4,sensorMeasure5,...,sensorMeasure12,sensorMeasure13,sensorMeasure14,sensorMeasure15,sensorMeasure16,sensorMeasure17,sensorMeasure18,sensorMeasure19,sensorMeasure20,sensorMeasure21
0,1.0,1.0,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392.0,2388.0,100.0,39.06,23.4190
1,1.0,2.0,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392.0,2388.0,100.0,39.00,23.4236
2,1.0,3.0,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390.0,2388.0,100.0,38.95,23.3442
3,1.0,4.0,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392.0,2388.0,100.0,38.88,23.3739
4,1.0,5.0,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393.0,2388.0,100.0,38.90,23.4044


In [10]:
df_test_fd001.head()

,unitNumber,cycle,operatingSet1,operatingSet2,operatingSet3,sensorMeasure1,sensorMeasure2,sensorMeasure3,sensorMeasure4,sensorMeasure5,...,sensorMeasure12,sensorMeasure13,sensorMeasure14,sensorMeasure15,sensorMeasure16,sensorMeasure17,sensorMeasure18,sensorMeasure19,sensorMeasure20,sensorMeasure21
0,1.0,1.0,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392.0,2388.0,100.0,38.86,23.3735
1,1.0,2.0,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393.0,2388.0,100.0,39.02,23.3916
2,1.0,3.0,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393.0,2388.0,100.0,39.08,23.4166
3,1.0,4.0,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391.0,2388.0,100.0,39.00,23.3737
4,1.0,5.0,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390.0,2388.0,100.0,38.99,23.4130


In [11]:
df_rul_fd001.head()

,RUL
0,112
1,98
2,69
3,82
4,91


In [12]:
# Aquí junto todos los dataset en uno solo

df_train = pd.concat([df_train_fd001, df_train_fd002, df_train_fd003, df_train_fd004],
                     ignore_index=True)

df_test = pd.concat([df_test_fd001, df_test_fd002, df_test_fd003, df_test_fd004],
                    ignore_index=True)

df_rul = pd.concat([df_rul_fd001, df_rul_fd002, df_rul_fd003, df_rul_fd004],
                   ignore_index=True)

In [13]:
print('Todo el dataset de entrenamiento tiene {} filas y {} atributos.'.format(*df_train.shape))
print('Todo el dataset de prueba tiene {} filas y {} atributos.'.format(*df_test.shape))
print('Todo el dataset de vida útil remanente tiene {} filas y {} atributo.'.format(*df_rul.shape))

Todo el dataset de entrenamiento tiene 160359 filas y 26 atributos.
Todo el dataset de prueba tiene 104897 filas y 26 atributos.
Todo el dataset de vida útil remanente tiene 707 filas y 1 atributo.


In [14]:
def modificar_dataset(df1, df2, bandera):
    
    """
    Esta función agrega dos columna, rul y rul_norm a cada dataset
    input: df1: Son los datos del dataset
           df2: Son los datos de los archivos RUL.
           bandera: valor de ayuda
    """
    
    if bandera == 1:
        # Crea un dataFrame con el máximo de ciclo por cada unidad
        rul = pd.DataFrame(df1.groupby('unitNumber')['cycle'].max()).reset_index()
        rul.columns = ['unitNumber', 'maxCycle']

        # Ahora insertamos esta columna con el dataframe original
        df1 = df1.merge(rul, on=['unitNumber'], how='left')

        # Se calcula el rul que se obtiene para cada unidad
        df1['rul'] = (df1['maxCycle'] - df1['cycle'])

        # Se calcula el rul normalizado que se obtiene para cada unidad
        df1['rul_norm'] = df1['cycle']/df1['maxCycle']

        # Se elimina la columna de ciclo máximo porque no se necesita para mas nada
        df1.drop('maxCycle', axis=1, inplace=True)
    
    if bandera == 2:

        # Crea un dataFrame con el maximo de ciclo por cada unidad
        rul = pd.DataFrame(df1.groupby('unitNumber')['cycle'].max()).reset_index()
        rul.columns = ['unitNumber', 'maxCycle']
        
        # Se suma el RUL del archivo RUL al ciclo máximo de cada unidad
        df2.columns = ['more']
        df2['unitNumber'] = df2.index + 1
        df2['maxCycle'] = rul['maxCycle'] + df2['more']
        df2.drop('more', axis=1, inplace=True)
        
        # Se calcula el rul que se obtiene para cada unidad
        # Se calcula el rul normalizado que se obtiene para cada unidad
        df1 = df1.merge(df2, on=['unitNumber'], how='left')
        df1['rul'] = df1['maxCycle'] - df1['cycle']
        df1['rul_norm'] = df1['cycle']/df1['maxCycle']
        
        # Se elimina la columna de ciclo maximo porque no se necesita para mas nada
        df1.drop('maxCycle', axis=1, inplace=True)
        
    return df1      

In [15]:
df_train_fd001 = modificar_dataset(df_train_fd001, df_rul_fd001, 1)
df_train_fd003 = modificar_dataset(df_train_fd003, df_rul_fd001, 1)
df_train_fd002 = modificar_dataset(df_train_fd002, df_rul_fd001, 1)
df_train_fd004 = modificar_dataset(df_train_fd004, df_rul_fd001, 1)

df_test_fd001 = modificar_dataset(df_test_fd001, df_rul_fd001, 2)
df_test_fd003 = modificar_dataset(df_test_fd003, df_rul_fd003, 2)
df_test_fd002 = modificar_dataset(df_test_fd002, df_rul_fd002, 2)
df_test_fd004 = modificar_dataset(df_test_fd004, df_rul_fd004, 2)

# Condiciones de operación

In [24]:
#Creo un dataframe que tenga todas las condiciones de operación únicas contenidas en el set de entrenamiento

sensors = [s for s in df_train.columns if "sensor" in s]
opset = [s for s in df_train.columns if "operating" in s]

# Crea un dataframe con las condiciones de operaciones unicas

df_opSet = df_train[opset].drop_duplicates().reset_index(drop=True)
df_opSet['wear'] = [df_opSet[opset].loc[i].tolist() for i in range(len(df_opSet.index))]

df_opSet.head()

,operatingSet1,operatingSet2,operatingSet3,wear
0,-0.0007,-0.0004,100.0,"[-0.0007, -0.0004, 100.0]"
1,0.0019,-0.0003,100.0,"[0.0019, -0.0003, 100.0]"
2,-0.0043,0.0003,100.0,"[-0.0043, 0.0003, 100.0]"
3,0.0007,0.0000,100.0,"[0.0007, 0.0, 100.0]"
4,-0.0019,-0.0002,100.0,"[-0.0019, -0.0002, 100.0]"


In [25]:
df_opSet.shape

(12289, 4)

In [26]:
df_opSet.to_csv('opset_data.csv')

In [27]:
def verifica(que, donde):
    
    """
    Esta función solo reporta la posicion del set de condiciones de operación
    """
    
    if que in donde:
        return donde.index(que)
    else:
        return float('NaN')

In [28]:
def opsetting(datos):
    
    Donde =  list(df_opSet['wear'].values)
    datos['opset_id'] = [verifica(datos[opset].loc[i].tolist(), Donde)
                        for i in list(range(len(datos.index)))]
    
    # Ordena las columnas que se desean

    cols = datos.columns.tolist()
    cols = cols[:5] + cols[-1:] + cols[5:-1]
    datos = datos[cols]
    
    # Elimina las columnas con las condiciones de operacion porque ya se tiene identificado
    datos.drop(opset, axis=1, inplace=True)
    
    return datos

In [29]:
# Entrenamiento
df_train_fd001 = opsetting(df_train_fd001)
df_train_fd003 = opsetting(df_train_fd003)
df_train_fd002 = opsetting(df_train_fd002)
df_train_fd004 = opsetting(df_train_fd004)

In [30]:
df_train_fd001.head()

,unitNumber,cycle,opset_id,sensorMeasure1,sensorMeasure2,sensorMeasure3,sensorMeasure4,sensorMeasure5,sensorMeasure6,sensorMeasure7,...,sensorMeasure14,sensorMeasure15,sensorMeasure16,sensorMeasure17,sensorMeasure18,sensorMeasure19,sensorMeasure20,sensorMeasure21,rul,rul_norm
0,1.0,1.0,0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,...,8138.62,8.4195,0.03,392.0,2388.0,100.0,39.06,23.4190,191.0,0.005208
1,1.0,2.0,1,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,...,8131.49,8.4318,0.03,392.0,2388.0,100.0,39.00,23.4236,190.0,0.010417
2,1.0,3.0,2,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,...,8133.23,8.4178,0.03,390.0,2388.0,100.0,38.95,23.3442,189.0,0.015625
3,1.0,4.0,3,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,...,8133.83,8.3682,0.03,392.0,2388.0,100.0,38.88,23.3739,188.0,0.020833
4,1.0,5.0,4,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,...,8133.80,8.4294,0.03,393.0,2388.0,100.0,38.90,23.4044,187.0,0.026042


In [34]:
# Prueba

df_test_fd001 = opsetting(df_test_fd001)
df_test_fd003 = opsetting(df_test_fd003)
df_test_fd002 = opsetting(df_test_fd002)
df_test_fd004 = opsetting(df_test_fd004)

In [35]:
df_test_fd001.shape

(13096, 26)

In [36]:
df_test_fd003.shape

(16596, 26)

In [37]:
df_test_fd002.shape

(33991, 26)

In [38]:
df_test_fd004.shape

(41214, 26)

In [39]:
df_test_fd001 = df_test_fd001[df_test_fd001['opset_id'].notnull()]
df_test_fd001 = df_test_fd001[df_test_fd001['opset_id'].notnull()]
df_test_fd001 = df_test_fd001[df_test_fd001['opset_id'].notnull()]
df_test_fd001 = df_test_fd001[df_test_fd001['opset_id'].notnull()]

In [40]:
df_test_fd001.shape

(13052, 26)

In [41]:
df_test_fd003.shape

(16596, 26)

In [42]:
df_test_fd002.shape

(33991, 26)

In [43]:
df_test_fd004.shape

(41214, 26)

In [44]:
df_train_fd001.to_csv('df_train_fd001.csv')
df_train_fd003.to_csv('df_train_fd003.csv')
df_train_fd002.to_csv('df_train_fd002.csv')
df_train_fd004.to_csv('df_train_fd004.csv')

df_test_fd001.to_csv('df_test_fd001.csv')
df_test_fd003.to_csv('df_test_fd003.csv')
df_test_fd002.to_csv('df_test_fd002.csv')
df_test_fd004.to_csv('df_test_fd004.csv')